In [ ]:
from analysis.analysis import *
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os

root = "corners_only_fixed"
gid = 1
save_path = f"../_server/sim_data/{root}/streamplots/grid {gid}/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

grid_df = read_fulltake(
    f"../_server/sim_data/corners_only_fixed/parquets/fulltake_gid{gid}.parquet"
)
grid_df = apply_time_filter(grid_df, dt=1.5 * grid_df.iloc[0].add_rate)


Compute original $F(x \rightarrow 0, v)$

In [ ]:
nbins = 32
delta = 2
min_pts = 2

xva_df = get_xva_df(grid_df, nbins=nbins)
bounds, F, sigma = compute_F_sigma(xva_df, nbins=nbins, min_pts=min_pts)

xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]

X_OG, Y_OG, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
F_v_x0_OG = F[:, nbins // 2 - delta : nbins // 2 + (delta + 1)].mean(axis=1)


Get bootstrapped samples
- each element contains a full set of runs as if it were a fulltake sim.


In [ ]:
from importlib import reload
from analysis import analysis

reload(analysis)
from analysis.analysis import *
from time import time

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=300)
mask = ~np.isnan(F_v_x0_OG)
ax.plot(Y_OG[mask, 0], F_v_x0_OG[mask], "-o", c="black")
ax.set_xlabel(r"$v$ ($\mu$m/hr)")
ax.set_ylabel(r"$F(x\rightarrow 0)$ ($\mu$m/hr$^2$)")

M = 20
for k in tqdm(range(M), total=M):
    sample_grid_df = bootstrap(
        grid_df, n_samples=grid_df.rid.unique().size, seed=int(time()) + k
    )
    sample_xva_df = get_xva_df(sample_grid_df, nbins=nbins)

    bounds, F, sigma = compute_F_sigma(sample_xva_df, nbins=nbins, min_pts=2)
    xmin, xmax = bounds["x"]
    vmin, vmax = bounds["v"]

    X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
    delta = 2
    F_v_x0 = F[:, nbins // 2 - delta : nbins // 2 + (delta + 1)].mean(axis=1)

    mask = ~np.isnan(F_v_x0)
    ax.plot(Y[mask, 0], F_v_x0[mask], c="black", alpha=0.1)

plt.show()